In [1]:
import cvxpy as cp
import numpy as np
# Generate a random problem np.random.seed(0)
m, n= 40, 25
A = np.random.rand(m, n)
b = np.random.randn(m)
# Construct a CVXPY problem
x = cp.Variable(n, integer=True)
objective = cp.Minimize(cp.sum_squares(A @ x - b)) 
prob = cp.Problem(objective)
prob.solve(solver = cp.GUROBI ,verbose = True)

                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) May 09 02:10:29 PM: Your problem has 25 variables, 0 constraints, and 0 parameters.
(CVXPY) May 09 02:10:29 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 09 02:10:29 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 09 02:10:29 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 09 02:10:29 PM: Compiling problem (target solver=GUROBI).
(CVXPY) May 09 02:10:29 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffi

13.746617913523945